In [69]:
import numpy as np
import random
import gymnasium as gym

In [70]:
env = gym.make("FrozenLake-v1", render_mode="human")

In [71]:
action_size = env.action_space.n
state_size = env.observation_space.n
qtable = np.zeros((state_size, action_size))
print(qtable)

[[0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]]


In [72]:
total_episodes = 50000        # Total episodes
learning_rate = 0.8           # Learning rate
max_steps = 99                # Max steps per episode
gamma = 0.95                  # Discounting rate

# Exploration parameters
epsilon = 1.0                 # Exploration rate
max_epsilon = 1.0             # Exploration probability at start
min_epsilon = 0.01            # Minimum exploration probability
decay_rate = 0.005             # Exponential decay rate for exploration prob

In [ ]:
rewards = []

for episode in range(total_episodes):
  # Reset environment + variables
  state = env.reset()[0]
  total_rewards = 0
  step = 0
  terminated = False

  for step in range(max_steps):
    tradeoff_value = random.uniform(0,1)
    # Exploit
    if tradeoff_value > epsilon:
      action = np.argmax(qtable[state,:])
    else:
      action = env.action_space.sample()

    # Execute action and observe results
    new_state, reward, terminated, info, done = env.step(action)

    # Update qtable
    # Q(s,a):= Q(s,a) + lr [R(s,a) + gamma * max Q(s',a') - Q(s,a)]
    qtable[state, action] = qtable[state, action] + learning_rate * (reward + gamma * np.max(qtable[new_state, :]) - qtable[state, action])

    state = new_state
    total_rewards += reward
    # If we're dead - finish episode prematurely
    if terminated:
      break

  # Reduce epsilon to increasingly discourage exploration as our table matures
  epsilon = min_epsilon + (max_epsilon - min_epsilon) * np.exp(-decay_rate * episode)
  rewards.append(total_rewards)

print ("Score over time: " +  str(sum(rewards)/total_episodes))
print(qtable)

In [ ]:
for episode in range(5):
    state = env.reset()[0]
    step = 0
    done = False
    print("****************************************************")
    print("EPISODE ", episode)

    for step in range(max_steps):

        # Take the action (index) that have the maximum expected future reward given that state
        action = np.argmax(qtable[state,:])

        new_state, reward, terminated, info, done = env.step(action)

        if done:
            # Here, we decide to only print the last state (to see if our agent is on the goal or fall into an hole)
            env.render()

            # We print the number of step it took.
            print("Number of steps", step)
            break
        state = new_state
env.close()
